```{contents}
```
## END Symbol

The **`END` symbol** in LangGraph represents the **terminal state** of execution.
It marks the point where the graph **halts**, returns the final state, and releases all runtime resources.

Formally, `END` is a **sentinel node** that defines **completion semantics** of the graph’s state machine.

---

### **1. Why `END` Exists**

LLM workflows are **not linear functions** — they are **state machines**.
Every state machine requires:

* a **start condition**
* a **termination condition**

`END` provides the **formal termination target**.

Without `END`, a graph has **no well-defined stopping point**.

---

### **2. Conceptual Role**

```
[Entry] → [Node A] → [Node B] → ... → [Node N] → END
```

When execution reaches `END`, LangGraph:

1. Freezes the current state
2. Returns it to the caller
3. Closes the execution context
4. Finalizes logs, checkpoints, and traces

---

### **3. How `END` Is Used**

### 3.1 Static Termination

```python
builder.add_edge("finalize", END)
```

Execution always ends after `finalize`.

---

### 3.2 Conditional Termination

```python
def router(state):
    if state["done"]:
        return END
    return "reason"

builder.add_conditional_edges(
    "observe",
    router,
    {"reason": "reason", END: END}
)
```

Termination becomes **state-dependent**.

---

### **4. `END` vs Normal Node**

| Property         | Normal Node | END |
| ---------------- | ----------- | --- |
| Executable       | Yes         | No  |
| Has handler      | Yes         | No  |
| Produces state   | Yes         | No  |
| Consumes state   | No          | No  |
| Terminates graph | No          | Yes |

`END` is **not a node** — it is an **execution target**.

---

### **5. `END` in Cyclic Graphs**

Cyclic graphs rely on `END` for safety:

```python
Reason → Act → Observe ─┐
        ↑               ↓
        └── until done ── END
```

`END` is the **only legal exit** from a loop.

Without it, the graph is **non-terminating**.

---

### **6. Execution Semantics**

When a transition reaches `END`:

* No further nodes are scheduled
* No further state updates are allowed
* Final state snapshot is emitted
* Checkpoint marked **complete**

---

### **7. Minimal Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    count: int

def increment(state):
    return {"count": state["count"] + 1}

def router(state):
    return END if state["count"] >= 3 else "inc"

builder = StateGraph(State)
builder.add_node("inc", increment)
builder.set_entry_point("inc")

builder.add_conditional_edges("inc", router, {"inc": "inc", END: END})

graph = builder.compile()
print(graph.invoke({"count": 0}))
```

**Execution Trace**

```
0 → 1 → 2 → 3 → END
```

---

### **8. Production Importance**

| Concern         | Role of `END`               |
| --------------- | --------------------------- |
| Resource safety | Frees memory & threads      |
| Cost control    | Stops token generation      |
| Reliability     | Prevents runaway graphs     |
| Auditability    | Defines completion boundary |
| Fault tolerance | Allows clean recovery       |

---

### **9. Design Rule**

> **Every LangGraph must have at least one reachable `END`.**

Graphs without reachable `END` are **invalid production systems**.

---

### **10. Mental Model**

`END` is equivalent to the **HALT instruction** in a CPU or the **accept state** in an automaton.

It defines **when the computation is complete**.


| Dimension                | **START**                            | **END**                                    |
| ------------------------ | ------------------------------------ | ------------------------------------------ |
| Definition               | Logical **entry point** of the graph | Logical **termination point** of the graph |
| Role                     | Begins execution                     | Halts execution                            |
| Type                     | Control symbol                       | Control symbol                             |
| Executable Node          | ❌ No                                 | ❌ No                                       |
| Has Handler              | ❌ No                                 | ❌ No                                       |
| Consumes State           | ❌ No                                 | ❌ No                                       |
| Produces State           | ❌ No                                 | ❌ No                                       |
| How It Is Set            | `builder.set_entry_point("node")`    | Referenced explicitly in edges             |
| Graph Location           | Only at beginning                    | Only at end                                |
| Number Allowed           | Exactly **one**                      | One or more reachable                      |
| Runtime Effect           | Initializes execution context        | Finalizes execution context                |
| Resource Management      | Allocates runtime                    | Releases runtime                           |
| Cost Control             | ❌                                    | ✔ Stops token usage                        |
| Loop Interaction         | Enters cycles                        | Exits cycles                               |
| Failure Handling         | Initializes recovery                 | Commits final checkpoint                   |
| Required for Valid Graph | ✔                                    | ✔                                          |
| Formal CS Analogy        | Program entry point                  | Program exit / HALT state                  |
| State Machine Analogy    | Initial state                        | Accept / terminal state                    |


In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    text: str

def process(state):
    return {"text": state["text"].upper()}

builder = StateGraph(State)

builder.add_node("process", process)
builder.set_entry_point("process")
builder.add_edge("process", END)

graph = builder.compile()

result = graph.invoke({"text": "hello"})
print(result)


{'text': 'HELLO'}


In [2]:
class State(TypedDict):
    count: int

def increment(state):
    return {"count": state["count"] + 1}

def router(state):
    if state["count"] >= 3:
        return END
    return "inc"

builder = StateGraph(State)

builder.add_node("inc", increment)
builder.set_entry_point("inc")

builder.add_conditional_edges(
    "inc",
    router,
    {"inc": "inc", END: END}
)

graph = builder.compile()
print(graph.invoke({"count": 0}))


{'count': 3}


In [3]:
class State(TypedDict):
    value: int
    done: bool

def work(state):
    v = state["value"] + 1
    return {"value": v, "done": v >= 5}

def route(state):
    return END if state["done"] else "work"

builder = StateGraph(State)

builder.add_node("work", work)
builder.set_entry_point("work")

builder.add_conditional_edges("work", route, {"work": "work", END: END})

graph = builder.compile()
print(graph.invoke({"value": 0, "done": False}))


{'value': 5, 'done': True}
